In [ ]:
### Задание 7.1: Декораторы для измерения времени
#Создайте декораторы для:
#- Измерения времени выполнения функции
#- Подсчета количества вызовов функции
#- Кэширования результатов функции (мемоизация)
#- Ограничения частоты вызовов функции (rate limiting)

In [ ]:
### Задание 7.2: Декораторы для валидации
#Реализуйте декораторы для:
#- Проверки типов аргументов функции
#- Валидации входных данных (например, email, телефон)
#- Проверки прав доступа (авторизация)
#- Логирования аргументов и результатов функции

In [ ]:
### Задание 7.3: Декораторы с параметрами
#Создайте декораторы, которые принимают параметры:
#- @retry(max_attempts=3, delay=1) - повторные попытки выполнения
#- @timeout(seconds=5) - ограничение времени выполнения
#- @deprecated(message="Use new_function instead") - пометка устаревших функций
#- @validate_input(schema=my_schema) - валидация по схеме

In [ ]:
### Задание 7.4: Комплексная система декораторов
#Создайте систему декораторов для API эндпоинтов:
#- @authenticated - проверка аутентификации
#- @rate_limited(requests_per_minute=60) - ограничение частоты запросов
#- @validate_json(schema) - валидация JSON
#- @cache(ttl=300) - кэширование ответов
#- @log_request - логирование запросов
#- Комбинируйте несколько декораторов на одной функции

In [ ]:
### Задание 8.1: Система управления задачами

import json
import pickle
import time
from collections import namedtuple, defaultdict, deque, OrderedDict, Counter, ChainMap
import os

# 1. namedtuple для задач
task = namedtuple('Task', ['id', 'title', 'description', 'priority', 'status', 'created_date', 'updated_date'])

# 2. кастомные исключения
class task_validation_error(Exception):
    pass

class task_not_found_error(Exception):
    pass

# 3. декоратор для логирования
def log_operation(func):
    def wrapper(*args, **kwargs):
        print(f"выполняется {func.__name__} с args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"завершено {func.__name__}")
        return result
    return wrapper

# глобальные переменные для хранения данных
tasks = []
next_id = 1
# группировка по статусу
tasks_by_status = defaultdict(list)
# очередь высокого приоритета
high_priority_queue = deque(maxlen=10)  
# счетчик приоритетов
priority_counter = Counter()
# история по порядку создания             
task_history = OrderedDict()             

status_list = {'todo', 'in_progress', 'done'}
priority_levels = {'low', 'medium', 'high'}
data_dir = 'task_data'

def validate_task_data(title, description, priority, status):
    """валидация данных задачи"""
    if not title or not description:
        raise task_validation_error("заголовок и описание не могут быть пустыми")
    if priority not in priority_levels:
        raise task_validation_error(f"неверный приоритет: {priority}")
    if status not in status_list:
        raise task_validation_error(f"неверный статус: {status}")

@log_operation
def add_task(title, description, priority='medium', status='todo'):
    """добавление новой задачи"""
    global next_id, tasks
    
    validate_task_data(title, description, priority, status)
    
    current_time = time.time()
    task_instance = task(
        id=next_id,
        title=title,
        description=description,
        priority=priority,
        status=status,
        created_date=current_time,
        updated_date=current_time
    )
    
    tasks.append(task_instance)
    _update_collections(task_instance)
    next_id += 1
    save_tasks()
    return task_instance

def _update_collections(task_instance):
    """обновление всех коллекций"""
    tasks_by_status[task_instance.status].append(task_instance)
    if task_instance.priority == 'high':
        high_priority_queue.appendleft(task_instance)
    priority_counter[task_instance.priority] += 1
    task_history[task_instance.created_date] = task_instance

@log_operation
def complete_task(task_id):
    """отметка задачи как выполненной"""
    return update_task_status(task_id, 'done')

@log_operation
def update_task_status(task_id, new_status):
    """обновление статуса задачи"""
    global tasks
    
    if new_status not in status_list:
        raise task_validation_error(f"неверный статус: {new_status}")
    
    for i, task_instance in enumerate(tasks):
        if task_instance.id == task_id:
            updated_task = task_instance._replace(
                status=new_status, 
                updated_date=time.time()
            )
            tasks[i] = updated_task
            _rebuild_collections()
            save_tasks()
            return updated_task
    
    raise task_not_found_error(f"задача с id {task_id} не найдена")

@log_operation
def remove_task(task_id):
    """удаление задачи"""
    global tasks
    
    for i, task_instance in enumerate(tasks):
        if task_instance.id == task_id:
            removed_task = tasks.pop(i)
            _rebuild_collections()
            save_tasks()
            return removed_task
    
    raise task_not_found_error(f"задача с id {task_id} не найдена")

def _rebuild_collections():
    """перестроение коллекций"""
    global tasks_by_status, high_priority_queue, priority_counter, task_history
    
    tasks_by_status.clear()
    priority_counter.clear()
    task_history.clear()
    high_priority_queue.clear()
    
    for task_instance in tasks:
        _update_collections(task_instance)

def get_tasks_by_status(status):
    """получение задач по статусу"""
    return tasks_by_status.get(status, [])

def get_priority_queue():
    """получение очереди высокоприоритетных задач"""
    return list(high_priority_queue)

def get_tasks_by_priority(priority):
    """получение задач по приоритету"""
    return [task_instance for task_instance in tasks if task_instance.priority == priority]

# 4. генератор для итерации по задачам
def task_generator(status_filter=None):
    """генератор для итерации по задачам"""
    for task_instance in tasks:
        if status_filter is None or task_instance.status == status_filter:
            yield task_instance

# 5. функциональное программирование для фильтрации
def filter_tasks(predicate):
    """фильтрация задач с использованием функционального подхода"""
    return list(filter(predicate, tasks))

def sort_tasks(key_func):
    """сортировка задач с использованием функционального подхода"""
    return sorted(tasks, key=key_func)

# работа с файлами через os.path
def _get_file_path(filename):
    return os.path.join(data_dir, filename)

def save_tasks():
    """сохранение в json и pickle"""
    os.makedirs(data_dir, exist_ok=True)
    tasks_data = [task_instance._asdict() for task_instance in tasks]
    
    # json
    with open(_get_file_path('tasks.json'), 'w', encoding='utf-8') as f:
        json.dump(tasks_data, f, ensure_ascii=False, indent=2)
    
    # pickle
    with open(_get_file_path('tasks.pickle'), 'wb') as f:
        pickle.dump(tasks_data, f)

def load_tasks():
    """загрузка из файлов"""
    global tasks, next_id
    
    json_file = _get_file_path('tasks.json')
    
    if os.path.exists(json_file):
        with open(json_file, 'r', encoding='utf-8') as f:
            tasks_data = json.load(f)
        
        tasks = [task(**data) for data in tasks_data]
        next_id = max([t.id for t in tasks], default=0) + 1
        _rebuild_collections()

def display_info():
    """краткая информация о задачах"""
    print(f"\nвсего задач: {len(tasks)}")
    print("по статусу:", {k: len(v) for k, v in tasks_by_status.items()})
    print("по приоритету:", dict(priority_counter))
    print("в очереди высокого приоритета:", len(high_priority_queue))

# chainmap для объединения различных списков задач
def get_task_chainmap():
    """создание chainmap для объединения списков задач"""
    return ChainMap(
        {'all_tasks': tasks},
        {'by_status': tasks_by_status},
        {'by_priority': {p: [t for t in tasks if t.priority == p] for p in priority_levels}},
        {'high_priority_queue': list(high_priority_queue)}
    )

# пример использования функционального программирования
def get_high_priority_todos():
    """получение высокоприоритетных задач в статусе todo"""
    return filter_tasks(lambda t: t.priority == 'high' and t.status == 'todo')

def sort_by_creation_date():
    """сортировка задач по дате создания"""
    return sort_tasks(lambda t: t.created_date)

In [ ]:
### Задание 8.2: Мини-игра "Крестики-нолики"
#Создайте игру "Крестики-нолики" с ИИ:
#- Класс GameBoard для игрового поля
#- Класс Player для игроков (человек и ИИ)
#- Класс Game для управления игрой
#- Используйте match-case для обработки состояний игры
#- Реализуйте простой ИИ с использованием генераторов
#- Добавьте логирование ходов и результатов
#- Обработайте все возможные ошибки

In [ ]:
### Задание 8.3: Система мониторинга файлов
#Создайте систему мониторинга изменений в файлах:
#- Класс FileMonitor для отслеживания файлов
#- Генератор для непрерывного мониторинга
#- Декораторы для логирования и кэширования
#- Обработка различных типов событий (создание, изменение, удаление)
#- Использование функционального программирования для обработки событий
#- Система уведомлений с кастомными исключениями